In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build

import numpy as np
import exafmm as fmm
import exafmm.modified_helmholtz as mod_helm

In [2]:
mod_helm.__doc__

"exafmm's submodule for Modified Helmholtz kernel"

### Create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

# generate random positions for particles
src_coords = np.random.random((nsrcs, 3))
trg_coords = np.random.random((nsrcs, 3))

# generate random charges for sources
src_charges = np.random.random(nsrcs)

In [4]:
# create a list of source instances
sources = mod_helm.init_sources(src_coords, src_charges)

# create a list of target instances
targets = mod_helm.init_targets(trg_coords)

### Create an LaplaceFMM instance

In [5]:
fmm = mod_helm.ModifiedHelmholtzFMM()
fmm.p = 10
fmm.nsurf = 6*(fmm.p-1)*(fmm.p-1) + 2
fmm.ncrit = 400
fmm.depth = 4
fmm.wavek = 10

### Setup FMM

Given sources, targets and FMM configurations, `setup()` builds the tree and interaction list and pre-compute invariant matrices.

Set `skip_P2P` to `True` to skip near-field interactions for BEM applications.

In [6]:
skip_P2P = False
tree = mod_helm.setup(sources, targets, fmm, skip_P2P)

### Evaluate

`evaluate()` triggers the evaluation and returns a $n_{trg} \times 4$ `numpy.ndarray`.
The $i$-th row of `trg_values` starts with the potential value of the $i$-th target, followed by its three gradient values.

In [7]:
trg_values = mod_helm.evaluate(tree, fmm)

P2M                  : 3.5960800e-01
M2M                  : 1.4360900e-01
P2L                  : 1.6990000e-03
M2P                  : 5.5500000e-04
P2P                  : 4.2238500e-01
M2L                  : 9.0301700e-01
L2L                  : 1.3491700e-01
L2P                  : 4.7320100e-01


In [8]:
trg_values[6]

array([4088.10980169, 4754.51633591, 8043.21328089, 2063.18225505])

### Check Accuracy

`fmm.verify(tree.leafs)` returns L2-norm the relative error of potential and gradient in a list. It only works when `skip_P2P` is set to `False`.

In [9]:
fmm.verify(tree.leafs)

[1.9377571638815348e-10, 1.1597814987925968e-08]

### Update charges of sources and run FMM iteratively

In [10]:
niters = 10

for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    
    src_charges = np.random.random(nsrcs)         # generate new random charges          
    mod_helm.update_charges(tree, src_charges)      # update charges
    mod_helm.clear_values(tree)                     # clear values
    trg_values = mod_helm.evaluate(tree, fmm)       # evaluate potentials

    print("Error: ", fmm.verify(tree.leafs))       # check accuracy

---------- iteration 0 ----------
P2M                  : 3.6971300e-01
M2M                  : 1.4016400e-01
P2L                  : 1.6890000e-03
M2P                  : 1.3330000e-03
P2P                  : 4.3144100e-01
M2L                  : 1.0010660e+00
L2L                  : 1.1197400e-01
L2P                  : 4.4744200e-01
Error:  [1.9877225849050226e-10, 1.942424207946107e-08]
---------- iteration 1 ----------
P2M                  : 3.6691700e-01
M2M                  : 1.8797500e-01
P2L                  : 6.8000000e-05
M2P                  : 5.7000000e-05
P2P                  : 4.2840900e-01
M2L                  : 8.7123100e-01
L2L                  : 1.1022400e-01
L2P                  : 4.6212400e-01
Error:  [2.2153124646038202e-10, 3.1652574995056185e-08]
---------- iteration 2 ----------
P2M                  : 3.5136200e-01
M2M                  : 1.2770600e-01
P2L                  : 1.6630000e-03
M2P                  : 3.1400000e-04
P2P                  : 4.2520700e-01
M2L     